In [2]:
K.<a> = NumberField(x - 1)
from sage.rings.polynomial.cyclotomic import cyclotomic_coeffs
S = ZZ['x']
K.<w> = NumberField(x^2 - x + 1)
R.<x> = PolynomialRing(ZZ)



def is_relatively_prime(m,n):
    if gcd(m,n) ==1:
        return 1
    else:
        return 0 
    
def raw_fekete(n):
    "Fekete polynomial with Delta odd "
    D=n
    v=[is_relatively_prime(a+1,n) for a in range(0,D-1)]
    F_D=R(v)
    return F_D*x  

def fekete_by_last_factor(n):
    F=raw_fekete(n)
    f = F.factor()[-1][0]
    return f

def divisor_set_general(q,p):
    divisor = set()
    for d in range(2, q):
        if q%d ==1:
            divisor.add(d)
    for d in range(2,p):
        if p%d==1 and d !=q:
            divisor.add(d)
    for d in range(2,p+q+1):
        if (q*p+1)%d==0 and (p+q)%d==0:
            divisor.add(d)
    return divisor 
   
    
def cyclotomic_factor(q,p):
    """
    return the cyclotmic factor of F_{qp}
    """
    cyc_factor =x
    divisor_set = divisor_set_general(q,p)
    for d in divisor_set:
        cyc_factor *=S(cyclotomic_coeffs(d))
    return cyc_factor


def fekete(q,p):
    n=p*q
    cyc_factor = cyclotomic_factor(q,p)
    F= raw_fekete(n)
    f,r =F.quo_rem(cyc_factor) 
    return f 


def reduced_fekete(f):
    u=f.trace_polynomial()
    g_D=u[0]
    return g_D

def fekete_reduction(f, q):
    f=f.change_ring(GF(q))
    return f.factor()        


def two_four_cycle(f,q):
    factor = fekete_reduction(f,q)
    number_of_factor = len(factor)
    #check that f is separable modulo q
    for i in range(number_of_factor):
        if factor[i][1] >1:
            return False
    number_two_cycle =0 
    number_four_cycle =0 
    number_even_cycle =0
    for i in range(number_of_factor):
        if factor[i][0].degree() ==2:
            number_two_cycle +=1
        if factor[i][0].degree() == 4:
            number_four_cycle +=1 
        if factor[i][0].degree() % 2 ==0:
            number_even_cycle +=1
    if number_two_cycle ==1 and number_four_cycle ==1 and number_even_cycle ==2:
        return True
    return False
            
    
def search_two_four_cycle(f,n):
    for q in range(n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 

def search_two_four_cycle_2(f,m,n):
    for q in range(m,n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 



def almost_cycle(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==3: 
                factor1=factor[0][0]
                factor2=factor[1][0]
                degree1=factor1.degree()
                degree2=factor2.degree()
                if degree1==1 and degree2==1 and factor[0][1]==1 and factor[1][1]==1 and factor[2][1]==1: 
                    return q
    return  -1         
                
def irreducible(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==1 and factor[0][1]==1:
                    return q
    return  -1         
                
       
    
def length_test_2(v):
    #count the number of even entries in v
    count2=0
    for item in v:
        if item==2:
            count2 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count2==count_even==1:
        return True
    return False    

def length_test_4(v):
    #count the number of even entries in v
    count4=0
    for item in v:
        if item==4:
            count4 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count4==count_even==1:
        return True
    return False    

    
def two_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1    

def two_cycle_2(f,m,n):
    result=[]
    for q in range(m,n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1 

def four_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_4(v):
            return q
    return -1  

def cycle(g,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(g,q)
            if len(factor)==2: 
                factor1=factor[0][0]
                coef=factor1.degree()
                if coef==1 and factor[0][1]==1 and factor[1][1]==1: 
                                   return q
    return  -1   
                    

                    
def search_quadruple(f,n):
    irr=irreducible(f,n)
    print(f"f is irreducible at q= ", irr)
    q_cycle=almost_cycle(f,n)
    print(f"f has an (2m-2) cycle at q=", q_cycle)
    q_tranposition=two_cycle(f,n)
    print(f"f has an 2-cycle at q=", q_tranposition)
    q_four_cycle=four_cycle(f,n)
    print(f"f has an 4-cycle at q=", q_four_cycle)
    

def quadruple(f,n):
    irr=irreducible(f,n)
    q_cycle=almost_cycle(f,n)
    q_tranposition=two_cycle(f,n)
    q_four_cycle=four_cycle(f,n)
    result=(irr, q_cycle, q_tranposition, q_four_cycle)
    return result
    
    
def triple(g,n):
    irr=irreducible(g,n)
    q_cycle=cycle(g,n)
    q_tranposition=two_cycle(g,n)
    result=[irr, q_cycle, q_tranposition]
    return result

In [13]:
q=5
p=7
n=q*p
F=raw_fekete(n)
print(F)

x^34 + x^33 + x^32 + x^31 + x^29 + x^27 + x^26 + x^24 + x^23 + x^22 + x^19 + x^18 + x^17 + x^16 + x^13 + x^12 + x^11 + x^9 + x^8 + x^6 + x^4 + x^3 + x^2 + x


In [14]:
f=fekete(q,p)
print(f)

x^22 - x^18 + x^17 - x^16 + x^15 - x^13 + x^12 + x^10 - x^9 + x^7 - x^6 + x^5 - x^4 + 1


In [15]:
g=reduced_fekete(f)
print(g)

x^11 - 11*x^9 + 43*x^7 + x^6 - 71*x^5 - 5*x^4 + 46*x^3 + 4*x^2 - 8*x + 2


In [17]:
fekete_reduction(g,29)

x^11 + 18*x^9 + 14*x^7 + x^6 + 16*x^5 + 24*x^4 + 17*x^3 + 4*x^2 + 21*x + 2

In [18]:
fekete_reduction(g,47)

(x + 6) * (x^10 + 41*x^9 + 25*x^8 + 38*x^7 + 3*x^6 + 30*x^5 + 31*x^4 + 44*x^3 + 17*x^2 + 43*x + 16)

In [19]:
fekete_reduction(g,31)

(x^2 + 13*x + 24) * (x^9 + 18*x^8 + 10*x^7 + 27*x^6 + 10*x^5 + 29*x^4 + 25*x^3 + 28*x^2 + 12*x + 13)

# Galois group of $g_{5p}$

In [21]:
P=Primes()
n=10**5
q=5
p=q
while p < 200: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

7 [29, 47, 31]
11 [113, 79, 197]


13 [23, 67, 241]
17 [19, 13, 7]
19 [229, 59, 103]


23 [79, 107, 113]


29 [1399, 163, 101]


31 [1091, 233, 29]
37 [293, 31, 3]


...

191 [127, 2161, 59]


193 [3329, 3463, 1439]


197 [313, 829, 307]


199 [739, 691, 449]


211 [149, 3449, 53]


In [22]:
%%time
P=Primes()
n=10**5
q=5
p=211
while p < 400: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

223 [3607, 1249, 19]


227 [103, 2677, 277]


229 [4127, 1277, 3]


233 [13, 4877, 29]


239 [277, 2309, 109]


...

379 [7481, 2999, 2473]


383 [3911, 1999, 421]


389 [9133, 9901, 349]


397 [1409, 1381, 67]


401 [17, 6911, 577]
CPU times: user 49min 26s, sys: 33.9 s, total: 50min
Wall time: 1h 44min 22s


In [23]:
%%time
P=Primes()
n=10**5
q=5
p=401
while p < 500: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

409 [6547, 673, 337]


419 [9239, 29027, 17]


421 [4289, 13781, 401]


431 [349, 13127, 257]


433 [2243, 5857, 479]


...

479 [11701, 2027, 23]


487 [5711, 20897, 691]


491 [30491, 1613, 193]


499 [9791, 3347, 359]


503 [73, 14887, 109]
CPU times: user 2h 24min 15s, sys: 1min 40s, total: 2h 25min 56s
Wall time: 5h 3min 59s


In [24]:
%%time
P=Primes()
n=10**5
q=5
p=500
while p < 600: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

503 [73, 14887, 109]


509 [6311, 1429, 601]


521 [8713, 1229, 661]


523 [1373, 12907, 887]


541 [3853, 21821, 101]


...

577 [3347, 9337, 373]


587 [10141, 10859, 1459]


593 [19, 33073, 839]


599 [7, 9397, 13]


601 [11317, 1459, 173]
CPU times: user 2h 38min 27s, sys: 1min 22s, total: 2h 39min 49s
Wall time: 5h 41min 2s


In [25]:
%%time
P=Primes()
n=10**5
q=5
p=601
while p < 700: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

607 [1213, 27239, 191]


613 [4513, 5827, 571]


617 [3019, 22189, 13]


619 [2557, 829, 701]


631 [10453, 16451, 151]


...

673 [4421, 1741, 23]


677 [4691, 12641, 347]


683 [11719, 17789, 17]


691 [24197, 5381, 29]


701 [5387, 19913, 179]
CPU times: user 5h 6min 28s, sys: 2min 47s, total: 5h 9min 15s
Wall time: 10h 36min 22s


In [26]:
%%time
P=Primes()
n=10**5
q=5
p=701
while p < 750: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

709 [3557, 3361, 359]


719 [49669, 30491, 443]


727 [8243, 571, 997]


733 [2549, 7541, 293]


739 [5981, 22013, 131]


743 [103, 9137, 109]


751 [11059, 1049, 1061]
CPU times: user 3h 25min 33s, sys: 1min 40s, total: 3h 27min 13s
Wall time: 7h 5min 52s


In [27]:
%%time
P=Primes()
n=10**5
q=5
p=751
while p < 800: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

757 [13469, 26713, 2423]


761 [28493, 7459, 863]


769 [11299, 7621, 443]


773 [27827, 30169, 887]


787 [4007, 167, 317]


797 [2339, 33757, 83]


809 [29137, 7927, 389]
CPU times: user 5h 17min 36s, sys: 2min 50s, total: 5h 20min 27s
Wall time: 5h 36min 58s


In [28]:
%%time
P=Primes()
n=10**5
q=5
p=809
while p < 850: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

811 [29569, 4349, 103]


821 [3109, 22807, 523]


823 [8123, 6301, 797]


827 [61043, 5527, 1871]


829 [12641, 14447, 23]


839 [31, 439, 1987]


853 [16141, 379, 3]
CPU times: user 4h 48min 33s, sys: 2min 51s, total: 4h 51min 25s
Wall time: 5h 5min 3s


In [29]:
%%time
P=Primes()
n=10**5
q=5
p=853
while p < 900: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

857 [17033, 587, 41]


859 [35543, 53681, 151]


863 [7673, 10243, 53]


877 [3917, 4349, 137]


881 [1931, 10159, 227]


883 [20287, 25717, 151]


887 [47737, 5647, 103]


907 [16901, 33749, 1759]
CPU times: user 8h 37min 12s, sys: 3min 28s, total: 8h 40min 40s
Wall time: 9h 13min 41s


In [30]:
%%time
P=Primes()
n=10**5
q=5
p=907
while p < 1000: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

911 [941, 15803, 227]


919 [73, 27031, 281]


929 [6271, 2797, 1031]


937 [-1, 11969, 2971]


941 [36187, 10273, 19]


...

977 [881, 8237, 2069]


983 [19213, 36571, 79]


991 [15877, 9463, 1063]


997 [7589, 19211, 347]


1009 [881, 101, 7]
CPU times: user 16h 57min 49s, sys: 11min 8s, total: 17h 8min 58s
Wall time: 18h 45min 4s


# Galois group of $f_{5p}$

We first consider the case $p \equiv 1 \pmod{4}$. In this case, the discriminant of $f_{5p}$ is not a perfect square.

In [34]:
P=Primes()
n=10**5
q=5
p=5
while p < 100: 
    p=P.next(p)
    if p % 4 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

13 67


17 431


29 967
37 31


41 157


53 3067


61 577


73 647


89 211


97 1103


101 4957


In [35]:
P=Primes()
n=10**5
q=5
p=101
while p < 300: 
    p=P.next(p)
    if p % 4 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

109 3739


113 1259


137 163


149 907


157 2267


...

257 2531


269 5527


277 487


281 1559


293 12671


In [36]:
%%time
P=Primes()
n=10**5
q=5
p=293
while p < 400: 
    p=P.next(p)
    if p % 4 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

313 13339


317 967


337 2087


349 7607


353 18143


373 7283


389 8867


397 7019


401 6911
CPU times: user 5h 31min 47s, sys: 3min 51s, total: 5h 35min 39s
Wall time: 5h 58min 17s


In [37]:
%%time
P=Primes()
n=10**5
q=5
p=401
while p < 500: 
    p=P.next(p)
    if p % 4 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

409 8291


421 16699


433 2371


449 8699


457 2971


461 4157
CPU times: user 5h 5min 4s, sys: 3min 1s, total: 5h 8min 6s
Wall time: 5h 33min 18s


In [38]:
%%time
P=Primes()
n=10**5
q=5
p=461
while p < 600: 
    p=P.next(p)
    if p % 4 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

509 691


521 6133


541 18911


557 8231


569 10391


577 9679


593 8171


601 6311
CPU times: user 15h 19min 14s, sys: 8min 12s, total: 15h 27min 27s
Wall time: 16h 52min 14s


In [39]:
%%time
P=Primes()
n=10**5
q=5
p=601
while p < 700: 
    p=P.next(p)
    if p % 4 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

613 8779


617 5807


641 16273


653 16223


661 14633


673 1439


677 25339


701 8219
CPU times: user 1d 6h 33min 57s, sys: 15min, total: 1d 6h 48min 58s
Wall time: 1d 12h 27min 46s


In [40]:
%%time
P=Primes()
n=10**5
q=5
p=701
while p < 750: 
    p=P.next(p)
    if p % 4 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

709 9103


733 13399
CPU times: user 10h 18min 27s, sys: 3min 39s, total: 10h 22min 6s
Wall time: 21h 53min 44s


We next consider the case $p \equiv 3 \pmod{4}$ but $p \equiv 1 \pmod{5}$. 

In [41]:
P=Primes()
n=10**5
q=5
p=5
while p < 100: 
    p=P.next(p)
    if p % 4 == 3 and p%5 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

11 53


31 317


71 587


In [43]:
%%time
P=Primes()
n=10**5
q=5
p=100 
while p < 300: 
    p=P.next(p)
    if p % 4 == 3 and p%5 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

131 1663


151 12983


191 5227


211 827


251 3347


271 857
CPU times: user 26min 17s, sys: 12.5 s, total: 26min 29s
Wall time: 56min 44s


In [44]:
%%time
P=Primes()
n=10**5
q=5
p=300
while p < 500: 
    p=P.next(p)
    if p % 4 == 3 and p%5 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

311 1493


331 3217


431 967


491 2633
CPU times: user 46min 4s, sys: 19.3 s, total: 46min 23s
Wall time: 1h 38min 11s


In [45]:
%%time
P=Primes()
n=10**5
q=5
p=500
while p < 600: 
    p=P.next(p)
    if p % 4 == 3 and p%5 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

571 3253
CPU times: user 42min 44s, sys: 16.4 s, total: 43min 1s
Wall time: 1h 30min 49s


In [46]:
%%time
P=Primes()
n=10**5
q=5
p=600
while p < 700: 
    p=P.next(p)
    if p % 4 == 3 and p%5 ==1:
        f = fekete(q,p)
        print(p, two_cycle(f,n))

631 16193


691 3463
CPU times: user 6h 39min 13s, sys: 2min 3s, total: 6h 41min 17s
Wall time: 13h 17min 9s


# The case $p \equiv 3 \pmod{4}$ and $p \not \equiv 1 \pmod{5}$

In [51]:
f = fekete(5,7)
fekete_reduction(f, 433)

(x + 97) * (x + 125) * (x^2 + 41*x + 1) * (x^4 + 124*x^3 + 295*x^2 + 124*x + 1) * (x^7 + 190*x^6 + 62*x^5 + 191*x^4 + 406*x^3 + 37*x^2 + 393*x + 313) * (x^7 + 289*x^6 + 393*x^5 + 76*x^4 + 168*x^3 + 50*x^2 + 251*x + 350)

In [50]:
%%time
P=Primes()
n=10**5
q=5
p=5
while p < 100: 
    p=P.next(p)
    if p % 4 == 3 and p%5 !=1:
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

7 433


19 1151


23 2437


43 7043


47 4831


59 7151


67 1949


79 971


83 13033
CPU times: user 3min 40s, sys: 2.03 s, total: 3min 42s
Wall time: 4min 8s


In [52]:
%%time
P=Primes()
n=10**5
q=5
p=100
while p < 300: 
    p=P.next(p)
    if p % 4 == 3 and p%5 !=1:
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

103 499


107 1979


127 8831


139 3709


163 293


...

227 38569


239 11437


263 10337


283 463


307 36721
CPU times: user 4h 53min 39s, sys: 1min 55s, total: 4h 55min 35s
Wall time: 9h 8min 59s


In [53]:
%%time
P=Primes()
n=10**5
q=5
p=307
while p < 400: 
    p=P.next(p)
    if p % 4 == 3 and p%5 !=1:
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

347 5477


359 4211


367 5869


379 8089


383 4951
CPU times: user 2h 20min 20s, sys: 34.8 s, total: 2h 20min 55s
Wall time: 4h 12min 46s


In [54]:
%%time
P=Primes()
n=10**5
q=5
p=400
while p < 500: 
    p=P.next(p)
    if p % 4 == 3 and p%5 !=1:
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

419 12841


439 15217


443 19681


463 5209


467 21323


In [3]:
%%time
P=Primes()
n=10**5
q=5
p=467
while p < 500: 
    p=P.next(p)
    if p % 4 == 3 and p%5 !=1:
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

479 -1


487 13043


499 6569


503 22697
CPU times: user 1d 1h 54min 8s, sys: 3min 19s, total: 1d 1h 57min 27s
Wall time: 2d 4h 27min 13s


In [6]:
q=5
p=479
m=10**5
n=2*m 
f = fekete(q,p)
print(p, search_two_four_cycle_2(f,m, n))

479 135211


# Sample example

In [7]:
q=5
p=7
f = fekete(5,7)
g=reduced_fekete(f)

In [8]:
g

x^11 - 11*x^9 + 43*x^7 + x^6 - 71*x^5 - 5*x^4 + 46*x^3 + 4*x^2 - 8*x + 2

In [10]:
fekete_reduction(f, 433)

(x + 97) * (x + 125) * (x^2 + 41*x + 1) * (x^4 + 124*x^3 + 295*x^2 + 124*x + 1) * (x^7 + 190*x^6 + 62*x^5 + 191*x^4 + 406*x^3 + 37*x^2 + 393*x + 313) * (x^7 + 289*x^6 + 393*x^5 + 76*x^4 + 168*x^3 + 50*x^2 + 251*x + 350)